In [ ]:
## imports
import pandas as pd
import tensorflow as tf
from os import *

In [4]:
## datasets, data cleanup, test-train split
ds = pd.read_csv('SPD_Crime_Data__2008-Present.csv')

## cleaning up the dates to have month, day, year columns
## also splitting it up into the layers of data
## which includes year, month, day

dateTime = ds['Offense Start DateTime']

dsYear = ds
dsMonth = ds
dsDay = ds

def getMonth(string):
    string = string[0:string.find(" ")]
    return string[0:string.find('/')]

def getDay(string):
    string = string[0:string.find(" ")]
    ph = string[0:string.find('/')+1]
    return ph[0:string.find('/')]

def getYear(string):
    string = string[0:string.find(" ")]
    ph = string[0:string.find('/')+1]
    pph = ph[0:string.find('/')+1]
    return pph[0:string.find('/')]

dsMonth.assign(Month = dateTime.apply(getMonth))

dsDay.assign(Day = dateTime.apply(getDay))

dsYear.assign(Year = dateTime.apply(getYear))

dsYear.loc[:, 'Crime Against Category':'Year']
dsMonth.loc[:, 'Crime Against Category':'Year']
dsDay.loc[:, 'Crime Against Category':'Year']

NameError: name 'pd' is not defined

In [ ]:
## train-test split
## to-do: for dsYear, Month, Day, create 3 splits
## each of 3 splits will be split 75% train and 25% test

In [ ]:
## polynomial regression
## to-do: learn how to do polynomial regression ,_,